In [1]:
import xarray as xr
import numpy as np
import sys, os, glob
import time
from scipy.stats import weibull_min
import dask.distributed as dd

root_dir = '/media/harish/SSD_4TB/EU_SCORES_project'
scripts_dir = f'{root_dir}/scripts'
sys.path.append(scripts_dir)

os.chdir(root_dir)

from data_processing.libraries import mean_statistics, std_statistics, wind_power_density, weibull , weibull_statistics, wind_power_density, turbine_power, solar_power
from data_processing.libraries import regional_extraction

root_dir = '/media/harish/SSD_4TB/EU_SCORES_project'
scripts_dir = f'{root_dir}/scripts'
sys.path.append(scripts_dir)

from analysis.plotters import map_plotter, all_sample_points, variability_plotter, months, colors, calculate_color_levels, CERRA_target_grids
from data_processing.libraries import longitude_convert_0_to_360, find_nearest_indice, compute_statistics

In [4]:
import dask.distributed as dd
cluster = dd.LocalCluster(n_workers=8, threads_per_worker=2, memory_limit='2GB',dashboard_address=f':8787')
client = dd.Client(cluster)
print(client)

<Client: 'tcp://127.0.0.1:43793' processes=8 threads=16, memory=14.90 GiB>


In [31]:
run = 'New_runs'
case = 'Ireland_coast'
points = [52.7466, -10.7352]
for level in [10,80,100,120,150]:
    run_dir=f'{root_dir}/WRFV4.4/EU_SCORES/{run}/{case}/Postprocessed/variablewise_files'
    ds = xr.open_dataset(f'{run_dir}/ws_{level}.nc')
    indices = find_nearest_indice(ds.XLAT,ds.XLONG,points[0],-10.51)
    ds.ws.sel(south_north=indices[0],west_east=indices[1]).assign_coords({'XLONG':points[1],'XLAT':points[0]}).to_netcdf(f'{run_dir}/sample_data/ws_{level}_{points[0]}_{points[1]}.nc')

Closest indices in the order of latitude (y) and longitude (x) are : (67, 11)
Closest indices in the order of latitude (y) and longitude (x) are : (67, 11)
Closest indices in the order of latitude (y) and longitude (x) are : (67, 11)
Closest indices in the order of latitude (y) and longitude (x) are : (67, 11)
Closest indices in the order of latitude (y) and longitude (x) are : (67, 11)
